Trying recipe from http://stackoverflow.com/questions/10628262/inserting-image-into-ipython-notebook-markdown

In [5]:
!ls

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


In [4]:
!pwd

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
pwd: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
pwd: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


In [1]:
print("helloworld")

helloworld


# Jupyter Version

<img src="jupyter-version.png">

# Script used

In [ ]:
// address of the notebook
var address = "http://localhost:8888/notebooks/phantomjs-tryout.ipynb"
// auth token from Jupyter console
var authToken = "b05a65d622136665a1292b9f3a29192364eaccb4bfbc6887"
// cell number with a widget output
var cellNumber = 1;

// this function is used to verify that a page is fully loaded
// source: https://github.com/ariya/phantomjs/blob/master/examples/waitfor.js
function waitFor(testFx, onReady, timeOutMillis) {
    var maxtimeOutMillis = timeOutMillis ? timeOutMillis : 3000,
        start = new Date().getTime(),
        condition = false,
        interval = setInterval(function() {
            if ( (new Date().getTime() - start < maxtimeOutMillis) && !condition ) {
                // If not time-out yet and condition not yet fulfilled
                condition = (typeof(testFx) === "string" ? eval(testFx) : testFx());
            } else {
                if(!condition) {
                    // If condition still not fulfilled (timeout but condition is 'false')
                    console.log("'waitFor()' timeout");
                    phantom.exit(1);
                } else {
                    // Condition fulfilled (timeout and/or condition is 'true')
                    console.log("'waitFor()' finished in " + (new Date().getTime() - start) + "ms.");
                    typeof(onReady) === "string" ? eval(onReady) : onReady();
                    clearInterval(interval); //< Stop this interval
                }
            }
        }, 250); //< repeat check every 250ms
};

// log in to a notebook using a token
function logIn() {
    console.log("Logging in");
    page.evaluate(function(token) {
        document.forms[0].password.value = token;
        document.forms[0].submit();
    }, authToken);
}

// wait for a notebook to fully load, find the
// needed output cell and save it as a PNG file
function saveAsPNG() {
    console.log("Saving PNG")
    // Wait for 'notebook-container' to be visible
    waitFor(function() {
        // Check in the page if a specific element is now visible
        return page.evaluate(function() {
            return $("#notebook-container").is(":visible");
        });
    }, function() {
        console.log("The notebook-container element should be visible now.");
        var clipRect = page.evaluate(function(cell){
            // we are selecting only the output cell
            var searchStr = 'div.output_area:contains("Out[' + cell + ']")'
            console.log('DEBUG: Search string: ' + searchStr);
            var outputArea = $(searchStr)[0];
            console.log('DEBUG: output_area div: ' + outputArea);
            // get only the data div
            var outputResult = $(outputArea).children('div.output_result')[0];
            console.log('DEBUG: output_result div: ' + outputResult);
            if (outputResult === undefined) {
                return -1
            }
            // get the coordinates of the data div
            return outputResult.getBoundingClientRect()
        }, cellNumber);

        console.log('DEBUG: clipRect: ' + clipRect);

        if (clipRect === -1) {
            phantom.exit(clipRect);
        }

        page.clipRect = {
            top:    clipRect.top,
            left:   clipRect.left,
            width:  clipRect.width,
            height: clipRect.height
          };
       page.render('example.png');
       phantom.exit();
    });
}

var page = require('webpage').create();
// it seems, viewportSize should fully cover the
// the rendered div position, or nothing will be saved.
page.viewportSize = { width: 5000, height: 5000 };

page.onConsoleMessage = function(msg, lineNum, sourceId) {
    console.log('CONSOLE: ' + msg);
}

page.open(address, function (status) {
    // Check for page load success
    if (status !== "success") {
        console.log("Unable to open a page");
    } else {
        console.log('DEBUG: NB address: ' + address)
        console.log('DEBUG: Auth token: ' + authToken)
        console.log('DEBUG: Cell number: ' + cellNumber)
        // Wait for 'password_input' to be visible
        waitFor(function() {
            // Check in the page if a specific element is now visible
            return page.evaluate(function() {
                return $("#password_input").is(":visible");
            });
        }, function() {
           console.log("The password_input element should be visible now.");
           logIn();
           saveAsPNG();
        });
    }
});

# Error

In [ ]:
DEBUG: NB address: http://localhost:8888/notebooks/test.ipynb
DEBUG: Auth token: b05a65d622136665a1292b9f3a29192364eaccb4bfbc6887
DEBUG: Cell number: 1
'waitFor()' finished in 500ms.
The password_input element should be visible now.
Logging in
Saving PNG
CONSOLE: loaded custom.js
CONSOLE: Default extension for cell metadata editing loaded.
CONSOLE: Raw Cell Format toolbar preset loaded.
CONSOLE: Slideshow extension for metadata editing loaded.
CONSOLE: actions jupyter-notebook:find-and-replace does not exist, still binding it in case it will be defined later...
CONSOLE: load_extensions [object Arguments]
CONSOLE: Loading extension: contrib_nbextensions_help_item/main
Failed to load extension: nbextensions/nbextensions_configurator/config_menu/main Error: Script error for "nbextensions/nbextensions_configurator/config_menu/main"
http://requirejs.org/docs/errors.html#scripterror

  http://localhost:8888/static/notebook/js/main.min.js?v=edbc43d9bb99b1be2585a121ed720314:12233
CONSOLE: Session: kernel_created (ef710647-5137-46c7-bd1e-cdc1778b0790)
CONSOLE: Starting WebSockets: ws://localhost:8888/api/kernels/b9b038d9-1801-4258-a62b-dc0e9b91d499
CONSOLE: Loading extension: toc2/main
CONSOLE: [toc2] Notebook fully loaded -- toc2 initialized 
CONSOLE: Widgets are not available.  Please install widgetsnbextension or ipywidgets 4.0
CONSOLE: ******Restoring toc display
'waitFor()' finished in 563ms.
The notebook-container element should be visible now.
CONSOLE: DEBUG: Search string: div.output_area:contains("Out[1]")
CONSOLE: DEBUG: output_area div: undefined
CONSOLE: DEBUG: output_result div: undefined
DEBUG: clipRect: -1


# Try with visualizer

In [6]:
# from https://github.com/yaroslavvb/notebooks/blob/master/simple_rewiring.ipynb
# make things wide
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def=None, width=1200, height=800, max_const_size=32, ungroup_gradients=False):
    if not graph_def:
        graph_def = tf.get_default_graph().as_graph_def()
        
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    data = str(strip_def)
    if ungroup_gradients:
        data = data.replace('"gradients/', '"b_')
        #print(data)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(data), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:{}px;height:{}px;border:0" srcdoc="{}"></iframe>
    """.format(width, height, code.replace('"', '&quot;'))
    display(HTML(iframe))


In [8]:
import tensorflow as tf
import numpy as np
a = tf.ones(())
b = tf.ones(())
c = a + b 
show_graph()

<img src="graphvis.png">